# SageMaker JumpStart 모델 온보딩 (Deep Java Library)


## ML 모델 패키징 프로세스

<img src="images/ml-model-publishing-workflow.png"/>

다음 다이어그램은 ML 모델 패키징 프로세스의 개요를 제공합니다.


- **1단계** 모델 아티팩트 및 서빙/스코어링 로직 저장
- **2단계** 추론을 수행하는 SageMaker에서 모델을 호스팅하는 데 사용되는 컨테이너를 생성하고 ECR에 푸시
- **3단계** SageMaker에서 모델을 성공적으로 호스팅할 수 있는 컨테이너 검증
- **4단계** ML 모델을 모델 패키지로 패키징
- **5단계** Amazon SageMaker에 배포하여 ML 모델 패키지 검증
- **6단계** AWS Marketplace에 ML 모델 등록

> **참고**: 모든 로컬 작업은 최적의 성능과 호환성을 위해 반드시 GPU가 지원되는 SageMaker Notebook 인스턴스에서 수행되어야 합니다.
> 
> [Deploy models with DJL Serving](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-models-frameworks-djl-serving.html)

In [ ]:
install_needed = True
# install_needed = False

In [ ]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install --upgrade pip --quiet
    !{sys.executable} -m pip install -U sagemaker --quiet
    !{sys.executable} -m pip install -U transformers --quiet
    IPython.Application.instance().kernel.do_shutdown(True)

# Start

In [1]:
%load_ext autoreload
%autoreload 2

### Model Store

In [2]:
import os
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
os.environ['HF_HOME'] = '/home/ec2-user/SageMaker/.cache'

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[04/03/25 05:20:29] INFO     Found credentials from IAM Role:                                   ]8;id=919387;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=994637;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


                    INFO     Found credentials from IAM Role:                                   ]8;id=136208;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=720947;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [3]:
from pathlib import Path

model_name_or_path='Qwen/Qwen2.5-7B-Instruct'
cache_dir = f'{Path.cwd()}/cache_dir'

In [4]:
import jinja2
jinja_env = jinja2.Environment()

<br>

## [**Step 1**] 모델 아티팩트 및 서빙/스코어링 로직 저장
---

### 모델 준비

In [5]:
import os

In [6]:
import transformers

In [7]:
def download_model(hf_model_id, local_model_path, ignore_patterns=None):
    import shutil
    from pathlib import Path
    from huggingface_hub import snapshot_download

    # 기본적으로 무시할 패턴 설정 (safetensors는 반드시 포함해야 함)
    if ignore_patterns is None:
        ignore_patterns = ["*.ckpt"]  # 큰 체크포인트 파일만 무시
    
    local_model_path = Path(local_model_path)
    print(f"Downloading model to: {local_model_path}")
    
    # 디렉토리 준비
    if os.path.exists(local_model_path):
        print(f"Model directory already exists. Updating files at: {local_model_path}")
    else:
        os.makedirs(local_model_path, exist_ok=True)
        print(f"Created new model directory: {local_model_path}")
    
    try:
        # 모델 다운로드
        allow_patterns = ["*.json", "*.safetensors", "*.pt", "*.txt", "*.model", "*.tiktoken", "*.gguf"]
        
        snapshot_download(
            repo_id=hf_model_id,
            local_dir=local_model_path,
            local_dir_use_symlinks=False,
            ignore_patterns=ignore_patterns,
            allow_patterns=allow_patterns,
        )
        
        # 다운로드 검증
        model_files = os.listdir(local_model_path)
        print(f"Downloaded files: {model_files}")
        
        # 필수 파일 확인
        if not os.path.exists(os.path.join(local_model_path, "config.json")):
            raise FileNotFoundError("config.json이 다운로드되지 않았습니다!")
        
        # 가중치 파일 확인
        weight_found = False
        for file in model_files:
            if file.endswith('.bin') or file.endswith('.safetensors'):
                weight_found = True
                break
                
        if not weight_found:
            raise FileNotFoundError("모델 가중치 파일이 다운로드되지 않았습니다!")
            
        print(f"모델 다운로드 완료: {hf_model_id}")
        return local_model_path
    
    except Exception as e:
        print(f"모델 다운로드 오류: {e}")
        raise

In [8]:
!rm -rf serve/checkpoint

In [9]:
model_name_or_path

'Qwen/Qwen2.5-7B-Instruct'

In [10]:
download_model(model_name_or_path, f"serve/checkpoint")

Created new model directory: serve/checkpoint


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/huggingface_hub/file_download.py:933: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

Downloaded files: ['model-00002-of-00004.safetensors', 'generation_config.json', 'tokenizer_config.json', 'vocab.json', '.cache', 'tokenizer.json', 'merges.txt', 'model-00004-of-00004.safetensors', 'model-00003-of-00004.safetensors', 'config.json', 'model-00001-of-00004.safetensors', 'model.safetensors.index.json']
모델 다운로드 완료: Qwen/Qwen2.5-7B-Instruct


PosixPath('serve/checkpoint')

#### model.tar.gz 구조 예시
다음은 model.tar.gz 구조의 예시입니다:
```
model_root_dir/              # 루트 디렉토리
├── serving.properties
├── model.py                 # Python을 위한 사용자 지정 핸들러 파일
└── 모델 바이너리 파일        # 모델 가중치 파일
```

In [11]:
%%writefile serve/serving.properties
option.model_id=/opt/ml/model/checkpoint
option.dtype=fp16
option.trust_remote_code=True
option.tensor_parallel_degree=max
option.gpu_memory_utilization=.95
option.max_model_len=10000
option.paged_attention=False
option.enable_streaming=False
option.kv_cache_dtype=fp8_e4m3

Overwriting serve/serving.properties


<br>

## [**Step 2**] 추론 수행을 위한 SageMaker 컨테이너 생성 및 ECR 등록
---

In [12]:
import boto3

region = boto3.Session().region_name

In [13]:
image_uri = "763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.32.0-lmi14.0.0-cu126"
docker_account_id = image_uri.split('.')[0]
print(f'image_uri: {image_uri} \ndocker_account_id : {docker_account_id}')

image_uri: 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.32.0-lmi14.0.0-cu126 
docker_account_id : 763104351884


In [14]:
!rm -rf docker && mkdir docker

In [15]:
%%writefile docker/Dockerfile

FROM 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.32.0-lmi14.0.0-cu126

ENV TZ=Asia/Seoul
RUN ln -snf /usr/share/zoneinfo/$TZ /etc/localtime && echo $TZ > /etc/timezone

ENV PYTHONUNBUFFERED=TRUE
ENV LANG C.UTF-8

Writing docker/Dockerfile


In [16]:
!rm -rf shell && mkdir shell

In [17]:
%%writefile shell/build_and_push.sh

# The name of our algorithm
algorithm_name={{algorithm_name}}
image_tag={{image_tag}}

cd {{root_dir}}/docker

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:${image_tag}"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin "{{docker_account_id}}.dkr.ecr.${region}.amazonaws.com"

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -f Dockerfile -t ${fullname} .

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

docker push ${fullname}

Writing shell/build_and_push.sh


In [18]:
algorithm_name='js-on-boarding-test'
image_tag='qwen-2-5-7b'

In [19]:
template = jinja_env.from_string(Path("shell/build_and_push.sh").open().read())
Path("shell/build_and_push.sh").open("w").write(template.render(algorithm_name=algorithm_name, image_tag=image_tag, root_dir=os.getcwd(), docker_account_id=docker_account_id))
# !pygmentize shell/build_and_push.sh | cat -n
!chmod +x shell/build_and_push.sh

In [20]:
!./shell/build_and_push.sh 

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
[+] Building 0.0s (0/1)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
[+] Building 0.2s (1/1)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 269B                                       0.0s
[+] Building 0.3s (1/1)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 269B                                       0.0s
[+] Building 0.5s (1/1)                                          docker:default
 => [internal] load build definition f

In [21]:
account = sagemaker.Session().account_id()
region = sagemaker.Session().boto_region_name

[04/03/25 05:20:59] INFO     Found credentials from IAM Role:                                   ]8;id=331610;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=424393;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=6386;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=505306;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [22]:
ecr_image_uri=f"{account}.dkr.ecr.{region}.amazonaws.com/{algorithm_name}:{image_tag}"
ecr_image_uri

'322537213286.dkr.ecr.us-west-2.amazonaws.com/js-on-boarding-test:qwen-2-5-7b'

<br>

## [**Step 3**] SageMaker에서 모델을 성공적으로 호스팅할 수 있는 컨테이너 검증
---

SageMaker 호스팅 엔드포인트로 배포하기 전에 로컬 모드 엔드포인트로 배포할 수 있습니다. 로컬 모드는 현재 개발 중인 환경에서 도커 컨테이너를 실행하여 SageMaker 프로세싱/훈련/추론 작업을 에뮬레이트할 수 있습니다. 추론 작업의 경우는 Amazon ECR의 딥러닝 프레임워크 기반 추론 컨테이너를 로컬로 가져오고(docker pull) 컨테이너를 실행하여(docker run) 모델 서버를 시작합니다.


### Uploading Model Data to S3

In [23]:
sagemaker_session = sagemaker.Session()

                    INFO     Found credentials from IAM Role:                                   ]8;id=507123;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=787854;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [24]:
bucket=sagemaker_session.default_bucket()
prefix='qwen-2-5-7B-instruct/model_data_tests'

In [25]:
!sudo rm -rf serve/.ipynb_checkpoints/
!sudo rm -rf serve/__pycache__/

In [26]:
import os

os.makedirs('shell', exist_ok=True)
os.makedirs('model', exist_ok=True)

In [27]:
%%writefile shell/pretrained_model_compression_upload.sh

cd serve
tar cvf - * | pigz > ../model.tar.gz

cd ..
mv model.tar.gz ./model/model.tar.gz

Writing shell/pretrained_model_compression_upload.sh


In [28]:
%%time
!sh ./shell/pretrained_model_compression_upload.sh

checkpoint/
checkpoint/model-00002-of-00004.safetensors
checkpoint/generation_config.json
checkpoint/tokenizer_config.json
checkpoint/vocab.json
checkpoint/.cache/
checkpoint/.cache/huggingface/
checkpoint/.cache/huggingface/download/
checkpoint/.cache/huggingface/download/vocab.json.lock
checkpoint/.cache/huggingface/download/model-00002-of-00004.safetensors.metadata
checkpoint/.cache/huggingface/download/model-00002-of-00004.safetensors.lock
checkpoint/.cache/huggingface/download/model-00004-of-00004.safetensors.metadata
checkpoint/.cache/huggingface/download/merges.txt.lock
checkpoint/.cache/huggingface/download/generation_config.json.metadata
checkpoint/.cache/huggingface/download/model.safetensors.index.json.metadata
checkpoint/.cache/huggingface/download/model.safetensors.index.json.lock
checkpoint/.cache/huggingface/download/model-00004-of-00004.safetensors.lock
checkpoint/.cache/huggingface/download/model-00001-of-00004.safetensors.metadata
checkpoint/.cache/huggingface/downloa

### SageMaker Endpoint (Local Mode)

로컬 모드는 필수로 수행할 필요는 없지만, 디버깅에 많은 도움이 됩니다. 또한, 로컬 모드 사용 시에는 모델을 S3에 반드시 업로드할 필요 없이 로컬 디렉터리에서도 로드할 수 있습니다. (`container` 변수 참조)

In [29]:
import boto3
import time

# Set to True to enable SageMaker to run locally
local_mode = True
# local_mode = False

if local_mode:
    from sagemaker.local import LocalSession
    instance_type = "local_gpu"
    sm_session = LocalSession()
    sm_client = sagemaker.local.LocalSagemakerClient()
    smr_client = sagemaker.local.LocalSagemakerRuntimeClient()
    model_data=f"file://{Path.cwd()}/model/model.tar.gz"
else:
    instance_type = "ml.g5.2xlarge"
    sm_session = sagemaker.Session()
    sm_client = boto3.client("sagemaker")
    smr_client = boto3.client("sagemaker-runtime")
    model_data = model_data_url

instance_count = 1
ts = time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
sm_model_name = f"qwen-model-{ts}"
endpoint_config_name = f"qwen-endpoint-config-{ts}"
endpoint_name = f"qwen-endpoint-{ts}"
model_data

[04/03/25 05:22:57] INFO     Found credentials from IAM Role:                                   ]8;id=343340;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=927279;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=48817;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=275301;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

'file:///home/ec2-user/SageMaker/jumpstart-onboarding/model/model.tar.gz'

In [30]:
env = {}
container = {
    "Image": ecr_image_uri,
    "ModelDataUrl": model_data,
    "Environment": env
}

In [31]:
create_model_response = sm_client.create_model(
    ModelName=sm_model_name, 
    ExecutionRoleArn=role, 
    PrimaryContainer=container,
)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": instance_type,
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": sm_model_name,
            "VariantName": "AllTraffic",
        }
    ],
)
#print("Model Arn: " + create_model_response["ModelArn"])

[04/03/25 05:22:58] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=740462;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=142739;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=468793;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=834463;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

In [32]:
!docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


In [33]:
!docker kill 695ddd1fefd5

Error response from daemon: Cannot kill container: 695ddd1fefd5: container 695ddd1fefd520d1e1bce21667bcc4b4f7aa27a5c60271e629c9a8aa217bb9a9 is not running


In [34]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)

                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=727416;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=798397;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     'Docker Compose' is not installed. Proceeding to check for                ]8;id=978768;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/local/image.py\image.py]8;;\:]8;id=611267;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/local/image.py#162\162]8;;\
                             'docker-compose' CLI.                                                                 

                    INFO     'Docker Compose' found using Docker Compose CLI.                          ]8;id=791618;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/local/image.py\image.py]8;;\:]8;id=30274;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/local/image.py#173\173]8;;\

                    INFO     serving                                                                   ]8;id=543419;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/local/image.py\image.py]8;;\:]8;id=73744;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/local/image.py#338\338]8;;\

                    INFO     creating hosting dir in /tmp/tmp4ie4b420                                  ]8;id=299180;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/local/image.py\image.py]8;;\:]8;id=124013;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/local/image.py#341\341]8;;\

...

                    WARNING  Retrying (Retry(total=1, connect=None, read=None, redirect=None, ]8;id=723211;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/urllib3/connectionpool.py\connectionpool.py]8;;\:]8;id=580723;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/urllib3/connectionpool.py#868\868]8;;\
                             status=None)) after connection broken by                                              
                             'ConnectionResetError(104, 'Connection reset by peer')': /ping                        

                    WARNING  Retrying (Retry(total=0, connect=None, read=None, redirect=None, ]8;id=211232;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/urllib3/connectionpool.py\connectionpool.py]8;;\:]8;id=92361;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/urllib3/connectionpool.py#868\868]8;;\
                             status=None)) after connection broken by                                              
                             'ConnectionResetError(104, 'Connection reset by peer')': /ping                        

                    INFO     Container still not up, got: -1                                        ]8;id=816736;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/local/entities.py\entities.py]8;;\:]8;id=452898;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/local/entities.py#998\998]8;;\

qh5f00rq0z-algo-1-889da  | INFO  PyProcess W-148-model-stdout: [1,0]<stdout>:INFO 04-03 14:26:04 model_runner.py:1523] Graph capturing finished in 16 secs.
qh5f00rq0z-algo-1-889da  | INFO  PyProcess Model [model] initialized.
qh5f00rq0z-algo-1-889da  | INFO  PyModel model model loaded in 48832 ms.
qh5f00rq0z-algo-1-889da  | INFO  WorkerPool scaling up min workers by 1 (from 0 to 1) workers. Total range is min 1 to max 1
qh5f00rq0z-algo-1-889da  | INFO  WorkerThread Starting worker thread WT-0001 for model model (M-0001, READY) on device gpu(0)
qh5f00rq0z-algo-1-889da  | INFO  ModelServer Initialize BOTH server with: EpollServerSocketChannel.
qh5f00rq0z-algo-1-889da  | INFO  ModelServer BOTH API bind to: http://0.0.0.0:8080


[04/03/25 05:26:08] INFO     Checking if serving container is up, attempt: 75                       ]8;id=528296;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/local/entities.py\entities.py]8;;\:]8;id=655778;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/local/entities.py#995\995]8;;\

In [35]:
!docker ps

CONTAINER ID   IMAGE                                                                          COMMAND                  CREATED              STATUS          PORTS                                       NAMES
2afd5ff2adc1   322537213286.dkr.ecr.us-west-2.amazonaws.com/js-on-boarding-test:qwen-2-5-7b   "/usr/local/bin/dock…"   About a minute ago   Up 55 seconds   0.0.0.0:8080->8080/tcp, :::8080->8080/tcp   qh5f00rq0z-algo-1-889da


### Inference Test

In [36]:
import json


prompt = "Give me a short introduction to large language model."

# 요청 데이터 준비
request_data = {
    "messages": [
        {"role": "user", "content": prompt}
    ],
    "max_new_tokens": 256  # 더 작은 값으로 제한
}


# API 호출
response = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Accept="application/json",
    ContentType="application/json",
    Body=json.dumps(request_data).encode('utf-8')
)


# 응답 처리
response_body = json.loads(response['Body'].read().decode('utf-8'))
print(response_body)


qh5f00rq0z-algo-1-889da  | INFO  PyProcess W-148-model-stdout: [1,0]<stdout>:INFO::[RequestId=9b1827d9-3513-423b-8c22-b5baafb654cb] parsed and scheduled for inference
{'id': 'chatcmpl-140172528068944', 'object': 'chat.completion', 'created': 1743657970, 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Certainly! A Large Language Model (LLM) is a type of artificial intelligence model that processes and generates human-like text. It is typically based on'}, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 39, 'completion_tokens': 30, 'total_tokens': 69}}


In [37]:
def delete_endpoint(client, endpoint_name):
    response = client.describe_endpoint(EndpointName=endpoint_name)
    EndpointConfigName = response['EndpointConfigName']
    
    response = client.describe_endpoint_config(EndpointConfigName=EndpointConfigName)
    model_name = response['ProductionVariants'][0]['ModelName']
    
    client.delete_model(ModelName=model_name)    
    client.delete_endpoint_config(EndpointConfigName=EndpointConfigName) 
    client.delete_endpoint(EndpointName=endpoint_name)
   
    print(f'--- Deleted model: {model_name}')
    print(f'--- Deleted endpoint_config: {EndpointConfigName}')     
    print(f'--- Deleted endpoint: {endpoint_name}')

In [38]:
delete_endpoint(sm_client, endpoint_name)
!sudo rm -rf /tmp/tmp*

Gracefully stopping... (press Ctrl+C again to force)
--- Deleted model: qwen-model-2025-04-03-05-22-57
--- Deleted endpoint_config: qwen-endpoint-config-2025-04-03-05-22-57
--- Deleted endpoint: qwen-endpoint-2025-04-03-05-22-57


### SageMaker Endpoint 에서 확인 (원격에서 호스팅)
> **⚠️ 주의** : 테스트 후 **반드시** 인스턴스 삭제가 필요합니다. 콘솔에서 SageMakerAI - Inference 에 SageMaker Endpoint가 삭제되었는지 확인해 주세요.

### Model을 S3로 업로드

In [39]:
tar_name = "./model/model.tar.gz"
model_data_url = sagemaker_session.upload_data(path=tar_name, bucket=bucket, key_prefix=prefix)

In [40]:
import boto3
import time

# Set to True to enable SageMaker to run locally
# local_mode = True
local_mode = False

if local_mode:
    from sagemaker.local import LocalSession
    instance_type = "local_gpu"
    sm_session = LocalSession()
    sm_client = sagemaker.local.LocalSagemakerClient()
    smr_client = sagemaker.local.LocalSagemakerRuntimeClient()
    model_data=f"file://{Path.cwd()}/model/model.tar.gz"
else:
    instance_type = "ml.g5.2xlarge"
    sm_session = sagemaker.Session()
    sm_client = boto3.client("sagemaker")
    smr_client = boto3.client("sagemaker-runtime")
    model_data = model_data_url

instance_count = 1
ts = time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
sm_model_name = f"qwen-model-{ts}"
endpoint_config_name = f"qwen-endpoint-config-{ts}"
endpoint_name = f"qwen-endpoint-{ts}"
model_data

[04/03/25 05:27:28] INFO     Found credentials from IAM Role:                                   ]8;id=556866;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=162227;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=171204;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=54830;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

's3://sagemaker-us-west-2-322537213286/qwen-2-5-7B-instruct/model_data_tests/model.tar.gz'

In [41]:
def invoke_endpoint(endpoint_name, request_data):
    import json
    
    response = smr_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Accept="application/json",
        ContentType="application/json",
        Body=json.dumps(request_data).encode('utf-8')
    )
    data = response["Body"].read()
    output = json.loads(data)
    return output

In [42]:
env = {
}

container = {
    "Image": ecr_image_uri,
    "ModelDataUrl": model_data,
    "Environment": env
}
container

{'Image': '322537213286.dkr.ecr.us-west-2.amazonaws.com/js-on-boarding-test:qwen-2-5-7b',
 'ModelDataUrl': 's3://sagemaker-us-west-2-322537213286/qwen-2-5-7B-instruct/model_data_tests/model.tar.gz',
 'Environment': {}}

In [43]:
create_model_response = sm_client.create_model(
    ModelName=sm_model_name, ExecutionRoleArn=role, PrimaryContainer=container
)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": instance_type,
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": sm_model_name,
            "VariantName": "AllTraffic",
        }
    ],
)
print("Model Arn: " + create_model_response["ModelArn"])
print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

Model Arn: arn:aws:sagemaker:us-west-2:322537213286:model/qwen-model-2025-04-03-05-27-28
Endpoint Config Arn: arn:aws:sagemaker:us-west-2:322537213286:endpoint-config/qwen-endpoint-config-2025-04-03-05-27-28


In [44]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

Endpoint Arn: arn:aws:sagemaker:us-west-2:322537213286:endpoint/qwen-endpoint-2025-04-03-05-27-28


In [45]:
from IPython.display import display, HTML
def make_console_link(region, endpoint_name, task='[SageMaker LLM Serving]'):
    endpoint_link = f'<b> {task} <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={region}#/endpoints/{endpoint_name}">Check Endpoint Status</a></b>'   
    return endpoint_link

endpoint_link = make_console_link(region, endpoint_name)
display(HTML(endpoint_link))

In [ ]:
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(30)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating


In [ ]:
def invoke_endpoint(endpoint_name, prompt):
    import json
    # 요청 데이터 준비
    request_data = {
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "max_new_tokens": 256  # 더 작은 값으로 제한
    }
    
    
    # API 호출
    response = smr_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Accept="application/json",
        ContentType="application/json",
        Body=json.dumps(request_data).encode('utf-8')
    )
    
    
    # 응답 처리
    response_body = json.loads(response['Body'].read().decode('utf-8'))
    print(response_body)
    return response_body


In [ ]:
prompt = "Give me a short introduction to large language model."

In [ ]:
%%time
res = invoke_endpoint(endpoint_name, prompt)

### Clean up

In [ ]:
def delete_endpoint(client, endpoint_name):
    response = client.describe_endpoint(EndpointName=endpoint_name)
    EndpointConfigName = response['EndpointConfigName']
    
    response = client.describe_endpoint_config(EndpointConfigName=EndpointConfigName)
    model_name = response['ProductionVariants'][0]['ModelName']
    
    client.delete_model(ModelName=model_name)    
    client.delete_endpoint_config(EndpointConfigName=EndpointConfigName) 
    client.delete_endpoint(EndpointName=endpoint_name)
   
    print(f'--- Deleted model: {model_name}')
    print(f'--- Deleted endpoint_config: {EndpointConfigName}')     
    print(f'--- Deleted endpoint: {endpoint_name}')

In [ ]:
delete_endpoint(sm_client, endpoint_name)

<br>

## [**Step 4**] ML 모델을 모델 패키지로 패키징
---
이 **step**에서는 아티팩트(ECR 이미지 및 학습된 모델 아티팩트)를 ModelPackage로 패키징하는 방법을 살펴봅니다. 이 작업을 완료하면 AWS 마켓플레이스에서 제품을 사전 학습된 모델로 등록할 수 있습니다.

**Note:** 모델을 여러 하드웨어 유형(CPU/GPU/Inferentia)에 배포할 수 있는 경우, 일반적으로 사용되는 컨테이너 이미지가 각각 다르기 때문에 각각에 대해 모델패키지를 생성하고 MP 목록에 다른 버전으로 추가해야 합니다.  

### 모델 패키지 사전 준비
모델 패키지는 추론에 필요한 모든 요소를 패키지로 묶은 모델 아티팩트에 대한 재사용 가능한 추상화 형태입니다. 이는 모델 데이터 위치(선택 사항)와 함께 사용할 추론 이미지를 정의하는 추론 사양으로 구성됩니다. ModelPackage는 AWS 마켓플레이스에 판매자로 등록할 AWS 계정에서 생성해야 합니다.

In [ ]:
import os
import boto3
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix='qwen-2-5-7B-instruct/model_data_tests'

region = sagemaker_session.boto_region_name
account= boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

s3_client = sagemaker_session.boto_session.client("s3")
sm_runtime = boto3.client("sagemaker-runtime")

In [ ]:
model_data = f's3://{bucket}/{prefix}/model.tar.gz'
model_data

In [ ]:
algorithm_name='js-on-boarding-test'
image_tag='qwen-2-5-7b'

In [ ]:
ecr_image_uri=f"{account}.dkr.ecr.{region}.amazonaws.com/{algorithm_name}:{image_tag}"
ecr_image_uri

In [ ]:
# <<YourSupportedContentTypes>>
supported_content_types = ["application/json"] #["text/csv", "application/json", "application/json", "application/jsonlines"]

# <<YourSupportedResponseMIMETypes>>
supported_response_MIME_types = [ 
    "application/json",
    # "text/csv",
    # "application/jsonlines",
]

supported_realtime_inference_instance_types = ["ml.g5.2xlarge"]
supported_batch_transform_instance_types = ["ml.g5.2xlarge"] #  Don't use batch transform. And, the Batch Transform validation step is not required

### 모델 패키지 생성
모델 패키지 생성 프로세스에서는 다음을 지정해야 합니다:
  1. 도커 이미지
  2. 모델 아티팩트
    - tar.gz 형태로 압축된 모델 아티팩트가 제공되어야 합니다.
        
판매자(및 구매자)에게 Amazon SageMaker에서 제품이 작동한다는 확신을 주기 위해, AWS Marketplace에 제품을 리스팅하기 전에 SageMaker는 기본적인 유효성 검사를 위와 같이 진행하였습니다. 이 유효성 검사 프로세스가 성공해야만 제품을 AWS Marketplace에 리스팅할 수 있습니다. 이 유효성 검사 프로세스는 사용자가 제공한 유효성 검사 프로필과 샘플 데이터를 사용하여 모델을 사용하여 계정에서 변환 작업을 생성하여 추론 이미지가 SageMaker에서 작동하는지 확인합니다.

다음으로, ML 모델에 적합한 인스턴스 크기를 식별해야 하며, ML 모델 위에서 성능 테스트를 실행하여 이를 확인할 수 있습니다.

**Note:** 모델 튜닝 외에도 인스턴스 유형을 식별할 때 모델의 요구 사항을 고려해야 합니다.  모델이 GPU 리소스를 사용하지 않는 경우 GPU 인스턴스 유형을 포함하지 마세요. 마찬가지로 모델이 GPU 리소스를 사용하지만 단일 GPU만 사용할 수 있는 경우, 여러 개의 GPU가 있는 인스턴스 유형을 포함하지 마세요. 성능상의 이점은 없이 사용자의 인프라 요금만 증가시킬 수 있기 때문입니다.

### 테스트용 샘플이미지 만들기

In [ ]:
prompt = "Give me a short introduction to large language model."
payload = {
    "messages": [
        {"role": "user", "content": prompt}
    ],
    "max_new_tokens": 256 
}

In [ ]:
from sagemaker.session import Session

sagemaker_session = Session()
s3_client = sagemaker_session.boto_session.client("s3")

In [ ]:
bucket = sagemaker_session.default_bucket()

In [ ]:
import json
json_line = json.dumps(payload)
with open("input.jsonl", "w") as f:
    f.write(json_line)
s3_client.put_object(Bucket=bucket, Key="validation-input-json/input.jsonl", Body=json_line)

In [ ]:
validation_file_name = "input.jsonl"
validation_input_path = f"s3://{bucket}/validation-input-json/"
validation_output_path = f"s3://{bucket}/validation-output-jsonl/"

### 패키지 생성

In [ ]:
# Define parameters
model_name = "marketplace-model-qwen-test-1" #"<<YourModelName>>"
model_description = "marketplace-model-qwen-test" #"<<YourModelDescription>>"

In [ ]:
model_package = sagemaker_session.sagemaker_client.create_model_package(
    ModelPackageName=model_name,
    ModelPackageDescription=model_description,
    InferenceSpecification={
        "Containers": [
            {
                "Image": ecr_image_uri,
                "ModelDataUrl": model_data
            }
        ],
        "SupportedTransformInstanceTypes": supported_batch_transform_instance_types,
        "SupportedRealtimeInferenceInstanceTypes": supported_realtime_inference_instance_types,
        "SupportedContentTypes": supported_content_types,
        "SupportedResponseMIMETypes": supported_response_MIME_types,
    },
    CertifyForMarketplace=True,  # Make sure to set this to True
   ValidationSpecification={
        'ValidationRole': role,
        'ValidationProfiles': [
            {
                'ProfileName': "validation",
                'TransformJobDefinition': {
                    'MaxConcurrentTransforms': 1,
                    'MaxPayloadInMB': 64,
                    'BatchStrategy': 'SingleRecord',
                    'TransformInput': {
                        'DataSource': {
                            'S3DataSource': {
                                'S3DataType': 'S3Prefix',
                                'S3Uri': f'{validation_input_path}input.jsonl'
                            }
                        },
                        'ContentType': 'application/json',
                        'CompressionType': 'None',
                        'SplitType': 'None'
                    },
                    'TransformOutput': {
                        'S3OutputPath': f'{validation_output_path}output.json',
                        'Accept': 'application/json',
                        'AssembleWith': 'None',
                    },
                    'TransformResources': {
                        'InstanceType': 'ml.g5.xlarge',
                        'InstanceCount': 1,
                    }
                }
            },
        ]
    },
)

In [ ]:
sagemaker_session.wait_for_model_package(model_package_name=model_name) # If failure occurs navigate to SageMaker Console > My marketplace model packages > select the failed ModelPackage for details. 

다음을 실행하기 전에, [Model Packages console from Amazon SageMaker](https://console.aws.amazon.com/sagemaker/home?region=us-east-1#/model-packages/my-resources)을 열어서 모델 생성의 성공했는지를 확인해야 합니다.
모델을 선택하고 **Validation** 탭을 열어서 validation 결과를 확인할 수 있습니다.

<br>

## [**Step 5**] Amazon SageMaker에 배포하여 ML 모델 패키지 검증
---

##### 모델 패키지에서 모델 객체 생성

In [ ]:
model_package['ModelPackageArn']

In [ ]:
from sagemaker import ModelPackage

model = ModelPackage(
    role=role,
    model_package_arn=model_package["ModelPackageArn"],
    sagemaker_session=sagemaker_session,
)

#### SageMaker 모델을 Endpoint로 배포

In [ ]:
model.deploy(
    initial_instance_count=1,
    instance_type=supported_realtime_inference_instance_types[0],
    endpoint_name=model_name,
)
model.endpoint_name

#### boto3로 예시 호출

In [ ]:
# Make use of your own example input data to test the Endpoint
import json


prompt = "Give me a short introduction to large language model."

# 요청 데이터 준비
request_data = {
    "messages": [
        {"role": "user", "content": prompt}
    ],
    "max_new_tokens": 256  # 더 작은 값으로 제한
}


response = sm_runtime.invoke_endpoint(
    EndpointName=model.endpoint_name,
    ContentType="application/json",
    Accept="application/json",
    Body=json.dumps(request_data),
)

json.load(response["Body"])

#### AWS CLI로 예시 호출

In [ ]:
sagemaker_session.boto_region_name

In [ ]:
# Perform inference
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name $model.endpoint_name \
    --body fileb://$validation_file_name \
    --content-type application/json \
    --region $sagemaker_session.boto_region_name \
    out.out
    
    
# Print inference
!head out.out

#### 생성된 endpoint configuration 과 endpoint 정리 

In [ ]:
model.sagemaker_session.delete_endpoint(model.endpoint_name)
model.sagemaker_session.delete_endpoint_config(model.endpoint_name)

- 이 모델은 필수가 아니므로 삭제해도 됩니다. 
- 배포 가능한 모델을 삭제한다는 점에 유의하세요. 
- 모델 패키지는 삭제하지 않습니다.

In [ ]:
model.delete_model()

##### AWS 마켓플레이스에 모델을 게시하려면 모델 패키지 ARN을 지정해야 합니다. 다음 모델 패키지 ARN을 복사합니다. 

In [ ]:
model_package["ModelPackageArn"]

<br>

## [**Step 6**] AWS Marketplace에 ML 모델 등록
---

1.  모델 파트너는 AWS 마켓플레이스에서 [public profile](https://docs.aws.amazon.com/marketplace/latest/userguide/seller-registration-process.html#seller-public-profile)을 생성하고 seller로 등록합니다.
마켓플레이스의 상품은 무료 상품으로 등록되므로 세금 정보를 제공할 필요가 없습니다.

2. 세이지메이커 콘솔의 [Model Packages](https://console.aws.amazon.com/sagemaker/home?region=us-east-1#/model-packages/my-resources) 섹션에서 이 노트북에서 생성한 엔티티를 찾을 수 있습니다. 성공적으로 생성되고 유효성이 검사되었다면 해당 엔티티를 선택하고 **Publish new ML Marketplace listing**를 선택할 수 있을 것입니다.

<img src="images/publish-to-marketplace-action.png"/>

리스팅을 작성할 수 있는 [AWS Marketplace Management portal](https://aws.amazon.com/marketplace/management/ml-products/)로 리디렉션됩니다.

<img src="images/listing.png"/>

1. 모델이 여러 하드웨어 유형을 대상으로 하는 경우 각 ModelPackage를 별도의 버전으로 목록에 추가하는 것을 잊지 마세요.
2. 추가를 클릭하고 모델 정보를 입력합니다. Product visibility을 'Public'로 설정해야 합니다.

<img src="images/public.png"/>

3. 테스트를 진행할 account 에 대해 모델 접근을 위한 Allowlist에 추가합니다. 예) account `171503325295`, `572320329544` and `559110549532` for access to the model. 
For region support select: `us-east-1, us-west-2, eu-west-1, eu-central-1, eu-west-2, ap-northeast-1, ap-south-1, ca-central-1, us-east-2, ap-northeast-2`
<img src="images/allowlist-accs.png"/>

4. Pricing and terms 하에 pricing 모델을 설정합니다.
**Inference based pricing (custom metering) at $0**

(선택 사항) 컨테이너가 아래를 구현하지 않은 경우 이를 확인하고 다음을 진행하세요. 

```
I confirm that my model package supports the response header for custom metering. Example response header: X-Amzn-Inference-Metering:
{"Dimension": "inference.count", "ConsumedUnits": 3}
I understand that in absence of this header, default metering will be used instead.
```

<img src="images/inference-based-pricing.png"/>

5. Listing 상태는 다음과 같이 표시되어야 합니다:
**Do not click Sign off and publish**

<img src="images/status-1.png"/>

6. Vissibility status of the listing should be `Limited`.

<img src="images/status-2.png"/>




**Resources**
* [Publishing your product in AWS Marketplace](https://docs.aws.amazon.com/marketplace/latest/userguide/ml-publishing-your-product-in-aws-marketplace.html)
